In [1]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install webdriver_manager

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install bs4 lxml

Defaulting to user installation because normal site-packages is not writeable


In [4]:
!pip install pymongo

Defaulting to user installation because normal site-packages is not writeable


In [5]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys

In [6]:
from bs4 import BeautifulSoup

In [7]:
driver = webdriver.Chrome(ChromeDriverManager().install())

/tmp/ipykernel_13399/1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [8]:
driver.get("https://edugate.ksu.edu.sa/ksu/ui/guest/timetable/index/mainScheduleTreeCoursesIndex.faces")

In [9]:
driver.find_element(By.ID,'myForm:timetableTreeId').click()

In [10]:
def has_setIndex(tag):
    return tag.has_attr('href') and 'setIndex' in tag['href']

In [11]:
source = driver.page_source
sp = BeautifulSoup(source, features='lxml')
tree = sp.find('div',{"class":"dtree"})

majors_dict = {}
for i in tree.find_all(has_setIndex):
    start = i['href'].find("'") + 1
    end = i['href'].rfind("'")
    majors_dict[(i['href'][start:end])] = i.text 

In [12]:
print("%5s" % "id:","Major Name")
for i, major in majors_dict.items():
    print("%5s" %i, ':', major)

  id: Major Name
0_0_0 : MRKN-التسويق
0_1_0 : IMNN-المالية
0_1_1 : IMN1-مسار المالية العام
0_2_0 : EC1-الاقتصاد العام
0_2_1 : ECN1-مسار الاقتصاد العام
0_2_2 : ECN2-مسار المصرفية الاسلامية
0_3_0 : PUAN-الإدارة
0_3_1 : PUN1-مسار الإدارة
0_3_2 : PUN2-مسار إدارة الموارد البشرية
0_4_0 : UMP01-إدارة الأعمال عام
0_5_0 : ACTN-المحاسبة
0_6_0 : INFN-نظم المعلومات الإدارية
1_0_0 : UPARB-لغة عربية
1_1_0 : MCPR-إعلام-علاقات عامه
1_1_1 : UPMC-إعلام (عام)
1_1_2 : AVM-مسار الإعلام المرئي والمسموع
1_1_3 : PPM-مسار الصحافة والنشر الإلكتروني
1_1_4 : SCMX-مسار العلاقات العامة
1_2_0 : UPHIS-تاريخ
1_3_0 : UPGEO-جغرافيا
1_3_1 : GOGD-مسار التنمية الإقليمية
1_3_2 : GOGM-مسار الخرائط ونظم المعلومات الجغرافية
1_4_0 : UPSC-علم الإجتماع
1_4_1 : UPPS-الخدمة الاجتماعية
1_5_0 : UPLIB-علم المعلومات
1_5_1 : LISM-مسار إدارة مصادر المعلومات وخدماتها
1_5_2 : LRME-مسار إدارة السجلات والحفظ الإلكتروني
1_6_0 : UPENG-اللغة الإنجليزية
2_0_0 : UPPO-العلوم السياسية
2_1_0 : UPLAW-الحقوق
3_0_0 : ICBL-دراسات اسلاميه (العقيدة)
3_0_1

In [13]:
def select_major(mId):
    mId = mId.strip()
    driver.execute_script(f"setIndex('{mId}')")

In [14]:
mId = input("Enter major id from the above table:")
while mId not in majors_dict:
    print("Wrong ID, try again!")
    mId = input("Enter major id from the above table:")

Enter major id from the above table:16_0_0


In [15]:
select_major(mId)

In [16]:
#Select search button
driver.find_element(By.XPATH,'//*[@id="myForm"]/table[4]/tbody/tr[1]/td/table/tbody/tr/td/table/tbody/tr[5]/td[6]/a[2]/table/tbody/tr/td[1]/img').click()

In [17]:
content = driver.page_source
soup = BeautifulSoup(content)
rt= soup.find(id="myForm:timetable")
trs = rt.find_all('tr')

In [18]:
def press_details(index=1):
    details_selector = '#myForm\:timetable > tbody > tr:nth-child({}) > td:nth-child(8) > a'
    details_button = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,details_selector.format(index))))
    details_button.click()

In [19]:
def press_details_js(index=0):
    driver.execute_script(f"showToolTipDilaog(event,'{index}');")

In [20]:
def get_times():
#     times = driver.find_element_by_id('sectionDiv').find_elements_by_tag_name('nobr')
    times = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.ID,'sectionDiv'))).find_elements(By.TAG_NAME,'nobr')
    if not times:
        return None
    n=len(times)//3
    return [times[i*3].text for i in range(n)], [times[1+i*3].text for i in range(n)] , [times[2+i*3].text for i in range(n)]

In [21]:
def get_final_info():
    return driver.find_element(By.ID,'examDayDiv').text,driver.find_element(By.ID,'examTimeDiv').text,driver.find_element(By.ID,'examDateDiv').text

In [22]:
def get_instructor():
    return driver.find_element(By.ID,'instructorDiv').text

In [23]:
def close():
    driver.find_element(By.XPATH,'//*[@id="dlg"]/div[1]/a/span').click()

In [24]:
# ignore the heading row
ctrs = trs[1:]

In [ ]:
# NOTE: the following code does not collect courses where the activity is "تدريب" or "مشروع".
# If you want to include thenm comment the if inside the loop
res = []
for x, i in enumerate(ctrs):
    tds = i.find_all('td')
    if 'مشروع' in tds[3].text or 'تدريب' in tds[3].text: continue
    tmp = {
        'cid': tds[0].text,
        'cname': tds[1].text,
        'section': tds[2].text,
        'activity': tds[3].text,
        'chours': tds[4].text,
        'gender': tds[5].text,
        'status': tds[6].text
    }
    press_details_js(x)
    driver.implicitly_wait(5)
    times = get_times()
    tmp['days'],tmp['time'],tmp['location']=(times) if times else ('','','')
    tmp['fexamday'],tmp['fexamtime'],tmp['fexamdate']=get_final_info()
    tmp['instructor'] = get_instructor()
    driver.implicitly_wait(5)
    close()
    driver.implicitly_wait(5)
    res.append(tmp)

In [ ]:
driver.quit()

In [ ]:
print("collected ",len(res),"courses.")

In [ ]:
import csv
keys = res[0].keys()

with open(f'{majors_dict[mId]}.csv', 'w',encoding='utf-8-sig', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(res)